In [1]:
#importing the files
import numpy as np
import pandas as pda
from sklearn.model_selection import train_test_split as tts
import matplotlib.pyplot as plt
from numpy.linalg import inv
import math
import itertools
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [2]:
microData = pda.read_csv('Microchip_Data.csv')

In [3]:
trainingSet,testingSet = tts(microData,test_size=0.3,random_state= 30)

In [4]:
trainingX = trainingSet[microData.columns[0:2]].copy()
trainingY = trainingSet['Pass/Fail'].copy()
testingX = testingSet[microData.columns[0:2]].copy()
testingY = testingSet['Pass/Fail'].copy()

In [5]:
trainingX = trainingX.to_numpy()
trainingY = trainingY.to_numpy()
testingX = testingX.to_numpy()
testingY = testingY.to_numpy()

In [6]:
addTraining = []
addTesting = []

for i in range(len(trainingX)):
    addTraining.append([trainingX[i][0]**2,trainingX[i][1]**2,trainingX[i][0]*trainingX[i][1]])
for i in range(len(testingX)):
    addTesting.append([testingX[i][0]**2,testingX[i][1]**2,testingX[i][0]*testingX[i][1]])

In [7]:
trainingX = np.concatenate((trainingX,addTraining),axis=1)
testingX = np.concatenate((testingX,addTesting),axis=1)

In [8]:
mini = np.min(trainingX,axis=0)
maxi = np.max(trainingX,axis=0)

In [9]:
trainingX = (trainingX - mini)/(maxi - mini)

In [10]:
mini = np.min(testingX,axis=0)
maxi = np.max(testingX,axis=0)

In [11]:
testingX = (testingX - mini)/(maxi - mini)

In [12]:
X_one = []
X_zero = []

for i in range(len(trainingY)):
    if trainingY[i] == 1:
        X_one.append([trainingX[i][0],trainingX[i][1],trainingX[i][2],trainingX[i][3],trainingX[i][4]])
    else:
        X_zero.append([trainingX[i][0],trainingX[i][1],trainingX[i][2],trainingX[i][3],trainingX[i][4]])

In [13]:
phi = float(len(X_one)/(len(X_one)+len(X_zero)))
mu0 = np.sum(np.matrix(X_zero),axis = 0)/len(X_zero)
mu1 = np.sum(np.matrix(X_one),axis = 0)/len(X_one)
print("Phi = ", phi)
print("mu0 = ", mu0)
print("mu1 = ", mu1)

Phi =  0.5365853658536586
mu0 =  [[0.51794166 0.51907225 0.34641179 0.3463046  0.47239801]]
mu1 =  [[0.49769095 0.50957137 0.19042111 0.18642523 0.43279346]]


In [14]:
sigma = np.zeros((5,5))
sigma0 = np.zeros((5,5))
sigma1 = np.zeros((5,5))

In [15]:
for i in range(len(trainingX)):
    if(trainingY[i] == 1):
        sigma1 += np.dot(np.transpose(trainingX[i]-mu1),trainingX[i]-mu1)
    else:
        sigma0 += np.dot(np.transpose(trainingX[i]-mu0),trainingX[i]-mu0)

In [16]:
sigma = (sigma1 + sigma0)/trainingX.shape[0]
sigma0 /= len(X_one)
sigma1 /= len(X_zero)
print("Sigma = ",sigma)
print("Sigma0 = ",sigma0)
print("Sigma1 = ",sigma1)

Sigma =  [[ 0.07182099 -0.00946001  0.021655    0.0012347   0.01438131]
 [-0.00946001  0.07938856 -0.01069037  0.0382644   0.01379159]
 [ 0.021655   -0.01069037  0.05704541 -0.02531739 -0.00519735]
 [ 0.0012347   0.0382644  -0.02531739  0.04885136 -0.00055653]
 [ 0.01438131  0.01379159 -0.00519735 -0.00055653  0.04646943]]
Sigma0 =  [[ 0.08148041 -0.00346765  0.0220916   0.0033302   0.01717463]
 [-0.00346765  0.09307716 -0.01168196  0.0474927   0.01817228]
 [ 0.0220916  -0.01168196  0.07798902 -0.03826915 -0.00955074]
 [ 0.0033302   0.0474927  -0.03826915  0.06078777  0.00397187]
 [ 0.01717463  0.01817228 -0.00955074  0.00397187  0.06253744]]
Sigma1 =  [[ 0.0606364  -0.01639852  0.02114947 -0.00119167  0.01114695]
 [-0.01639852  0.06353861 -0.00954222  0.02757901  0.00871921]
 [ 0.02114947 -0.00954222  0.03279492 -0.0103206  -0.00015657]
 [-0.00119167  0.02757901 -0.0103206   0.03503026 -0.00579993]
 [ 0.01114695  0.00871921 -0.00015657 -0.00579993  0.02786436]]


In [17]:
def probFunction(x,mu,sigma):
    m = len(x)
    if m == mu.shape[1] and (m,m) == sigma.shape:
        deter = np.linalg.det(sigma)
        assert deter!=0, "matrix cannot be singular"
        
        temp = 1.0/(np.power((2*np.pi),float(m)/2)*np.power(deter,1.0/2))
        xmu = np.matrix(x-mu)
        siginv = inv(sigma)
        res = np.power(np.e,-0.5*(np.dot(np.dot(xmu,siginv),np.transpose(xmu))))
        return res*temp


In [18]:
predictedY = []
for x in testingX:
    pa = probFunction(x,np.squeeze(mu0),np.matrix(sigma0))
    pc = probFunction(x,np.squeeze(mu1),np.matrix(sigma1))
    if (pa<pc):
        predictedY.append(1)
    else:
        predictedY.append(0)

In [19]:
count = 0
for i in range(len(testingY)):
    if(predictedY[i]==testingY[i]):
        count = count + 1

accuracy = np.multiply(np.divide(count,len(testingY)),100)
print(accuracy)

88.88888888888889


In [20]:
accuracy_score(testingY,predictedY)

0.8888888888888888

In [21]:
confusion_matrix(testingY,predictedY)

array([[18,  4],
       [ 0, 14]])